# Truck transportation industry overview - data cleaning.

I have gathered publicly available historical data about factors impacting trucking industry in the United States which I consider important for business operation and strategy planning. I will focus on data for period 2006-2019.

## GOAL

#### The goal of this notebook is to clean the data and prepare for exploratory data analysis and dashboard creation.<br><br>
#### Steps to acheive the goal look as follows: <br><br>
1.Loading the data.<br>
2.Summarizing and describing the data.
3.Identyfing duplicate values and deleting them.<br>
4.Checking for missing values and deciding on the approach towards them.<br>
5.Transforming the data into desired shape.<br>
6.Validating the data.<br>
7.Exporting the data.<br>


### Tools 
To achieve the goal I have decided to use Python and the libraries listed below. The files are in csv format.

### Note 
I suggest getting familiar with the readme file to obtain more information about the datasets and its sources. 

In [58]:
import pandas as pd
import numpy as np
import glob

I will use glob and pandas libraries in order to quickly load 11 datasets.

### Loading the data

In [59]:
path = r'C:\Users\Tymoteusz\Desktop\Truck data US\csvs'
files = glob.glob(path + '\*.csv')

df_list = []

for f in files:
    temporary_df = pd.read_csv(f)
    df_list.append(temporary_df)
    print(temporary_df.head(),temporary_df.describe(),temporary_df.info())

# loading the datasets to the list and displaying summary stats


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40 entries, 0 to 39
Data columns (total 39 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Category  6 non-null      object 
 1   1960      6 non-null      object 
 2   1965      6 non-null      object 
 3   1970      6 non-null      object 
 4   1975      6 non-null      object 
 5   1980      6 non-null      object 
 6   1985      6 non-null      object 
 7   1990      6 non-null      float64
 8   1991      6 non-null      float64
 9   1992      6 non-null      float64
 10  1993      6 non-null      float64
 11  1994      6 non-null      float64
 12  1995      6 non-null      float64
 13  1996      6 non-null      float64
 14  1997      6 non-null      float64
 15  1998      6 non-null      float64
 16  1999      6 non-null      float64
 17  2000      6 non-null      float64
 18  2001      6 non-null      float64
 19  2002      6 non-null      float64
 20  2003      6 non-null      object 


[5 rows x 42 columns]                               Categories       1980       1981       1982  \
count                                 12         12         12         12   
unique                                12         12         12         12   
top     TOTAL U.S. ton-miles of freight   4,207,610  4,230,548  4,122,111   
freq                                   1          1          1          1   

             1983       1984       1985       1986       1987       1988  ...  \
count          12         12         12         12         12         12  ...   
unique         12         12         12         12         12         12  ...   
top     4,013,105  4,317,065  4,342,117  4,360,857  4,409,101  4,414,966  ...   
freq            1          1          1          1          1          1  ...   

         (R) 2011   (R) 2012   (R) 2013   (R) 2014   (R) 2015   (R) 2016  \
count          12         12         12         12         12         12   
unique         12         12      

I have decided to display some descriptive and summary statistics about columns of each dataset in order to get more insights about the values. The statistics are helpful in determining the data types and show us inconsistencies in the data types among the columns which might be due to missing or incorrectly entered values.

As we can see some of the datasets have the "Year" values entered as separate columns while the others have it stored in single column. We will have to address this inconsistency later. We can also notice a lot of missing values in some of the datasets. Some of them exist, due to the way the datasets were converted before loading.

I will create functions to automate some of the data cleaning tasks.

### Checking for duplicates

In [60]:
def duplicate_rows(df):
    print(df[df.duplicated()])
# The function prints dataframe of duplicated rows in provided dataframe    

In [61]:
def execute_function(list_df,func,func_desc):
    print(func_desc)
    for i in range(len(list_df)):
        func(list_df[i])
        print(list_df[i].head())
        
execute_function(df_list,duplicate_rows,'Duplicate rows')
# Loops through the list and applies provided function to the dataframes, takes list of dfs, function to execute on them and a descriptive value as a string to be displayed at first.

Duplicate rows
   Category 1960 1965 1970 1975 1980 1985  1990  1991  1992  ...  2012  2013  \
7       NaN  NaN  NaN  NaN  NaN  NaN  NaN   NaN   NaN   NaN  ...   NaN   NaN   
8       NaN  NaN  NaN  NaN  NaN  NaN  NaN   NaN   NaN   NaN  ...   NaN   NaN   
9       NaN  NaN  NaN  NaN  NaN  NaN  NaN   NaN   NaN   NaN  ...   NaN   NaN   
10      NaN  NaN  NaN  NaN  NaN  NaN  NaN   NaN   NaN   NaN  ...   NaN   NaN   
11      NaN  NaN  NaN  NaN  NaN  NaN  NaN   NaN   NaN   NaN  ...   NaN   NaN   
12      NaN  NaN  NaN  NaN  NaN  NaN  NaN   NaN   NaN   NaN  ...   NaN   NaN   
13      NaN  NaN  NaN  NaN  NaN  NaN  NaN   NaN   NaN   NaN  ...   NaN   NaN   
14      NaN  NaN  NaN  NaN  NaN  NaN  NaN   NaN   NaN   NaN  ...   NaN   NaN   
15      NaN  NaN  NaN  NaN  NaN  NaN  NaN   NaN   NaN   NaN  ...   NaN   NaN   
16      NaN  NaN  NaN  NaN  NaN  NaN  NaN   NaN   NaN   NaN  ...   NaN   NaN   
17      NaN  NaN  NaN  NaN  NaN  NaN  NaN   NaN   NaN   NaN  ...   NaN   NaN   
18      NaN  NaN  NaN  Na

After checking for duplicate rows in the dataframes I have noticed that the only rows which are duplicated are rows containing NaN values. We will get rid of them in the step where we handle missing values.

### Handling missing values and transformation

Before applying the functions we need to transform some of the values containing strings into NaN. <br><br> In all of the datasets missing values were denoted as 'S', 'N' and 'U', based on the reason behind the missingness of given value. The NaN values are mostly empty cells from excel which remained in the table after conversion to the csv file.<br> <br>More information about the original datasets can be found in the sources provided in the Readme file.<br><br> We need to make sure that we include only columns which should contain numeric values. Considering the reasons behind the missingness of the data we can safely delete missing values and we do not risk missing valuable information.

In [62]:
def missing_to_nan(df):
    df.replace(['N','S','U'],np.nan,inplace=True)
    print(df[df.isna().any(axis=1)])
    
    # function converts missing values encoded as N, S or U to np.nan.

In [63]:
def count_missing(df):
    a = 0
    for col in df.columns:
        
        a += df[col].astype(str).str.contains('U').sum()
        a += df[col].astype(str).str.contains('N').sum()
        a += df[col].astype(str).str.contains('S').sum()
    print('U,N,S counts')
    print(a)
# function counts the missing values encoded as U, N or S.

In [64]:
execute_function(df_list,count_missing,'Number of missing values')

Number of missing values
U,N,S counts
27
                         Category   1960   1965   1970   1975   1980   1985  \
0          Air carrier, domestica  22.80  20.46  21.91  28.22  46.31  48.77   
1                          Truckb      N      N      N      N      N   5.34   
2                    Class I rail   1.40   1.27   1.43   2.04   2.87   3.04   
3  Water Transportation, domestic      N      N      N      N      N      N   
4                    Oil pipeline      N      N      N      U      N      N   

    1990   1991   1992  ...    2012    2013    2014    2015    2016    2017  \
0  59.71  60.95  63.07  ...  133.51  135.47  135.85  124.04  119.90  125.15   
1   7.64   7.54   7.75  ...   16.72   15.99   17.68   17.75   17.34   16.10   
2   2.66   2.59   2.58  ...    3.95    4.05    4.05    3.97    3.99    4.02   
3   1.42   1.41   1.39  ...    3.18    3.36    3.37    3.25    3.00    2.74   
4   1.22   1.18   1.21  ...    1.63    1.92    2.26    2.49    2.66    2.88   

     2018

In [65]:
def display_nan(df):
    print(df[df.isna().any(axis=1)])
    print(df)
    # This function displays NaN values in the provided df

In [66]:
execute_function(df_list,display_nan,'Missing values')

Missing values
   Category 1960 1965 1970 1975 1980 1985  1990  1991  1992  ...  2012  2013  \
6       NaN  NaN  NaN  NaN  NaN  NaN  NaN   NaN   NaN   NaN  ...   NaN   NaN   
7       NaN  NaN  NaN  NaN  NaN  NaN  NaN   NaN   NaN   NaN  ...   NaN   NaN   
8       NaN  NaN  NaN  NaN  NaN  NaN  NaN   NaN   NaN   NaN  ...   NaN   NaN   
9       NaN  NaN  NaN  NaN  NaN  NaN  NaN   NaN   NaN   NaN  ...   NaN   NaN   
10      NaN  NaN  NaN  NaN  NaN  NaN  NaN   NaN   NaN   NaN  ...   NaN   NaN   
11      NaN  NaN  NaN  NaN  NaN  NaN  NaN   NaN   NaN   NaN  ...   NaN   NaN   
12      NaN  NaN  NaN  NaN  NaN  NaN  NaN   NaN   NaN   NaN  ...   NaN   NaN   
13      NaN  NaN  NaN  NaN  NaN  NaN  NaN   NaN   NaN   NaN  ...   NaN   NaN   
14      NaN  NaN  NaN  NaN  NaN  NaN  NaN   NaN   NaN   NaN  ...   NaN   NaN   
15      NaN  NaN  NaN  NaN  NaN  NaN  NaN   NaN   NaN   NaN  ...   NaN   NaN   
16      NaN  NaN  NaN  NaN  NaN  NaN  NaN   NaN   NaN   NaN  ...   NaN   NaN   
17      NaN  NaN  NaN  Na

[40 rows x 39 columns]
                         Category   1960   1965   1970   1975   1980   1985  \
0          Air carrier, domestica  22.80  20.46  21.91  28.22  46.31  48.77   
1                          Truckb      N      N      N      N      N   5.34   
2                    Class I rail   1.40   1.27   1.43   2.04   2.87   3.04   
3  Water Transportation, domestic      N      N      N      N      N      N   
4                    Oil pipeline      N      N      N      U      N      N   

    1990   1991   1992  ...    2012    2013    2014    2015    2016    2017  \
0  59.71  60.95  63.07  ...  133.51  135.47  135.85  124.04  119.90  125.15   
1   7.64   7.54   7.75  ...   16.72   15.99   17.68   17.75   17.34   16.10   
2   2.66   2.59   2.58  ...    3.95    4.05    4.05    3.97    3.99    4.02   
3   1.42   1.41   1.39  ...    3.18    3.36    3.37    3.25    3.00    2.74   
4   1.22   1.18   1.21  ...    1.63    1.92    2.26    2.49    2.66    2.88   

     2018       2019    202

         DATE  PCU484484
0  2003-12-01      100.0
1  2004-01-01      100.4
2  2004-02-01      101.1
3  2004-03-01      101.3
4  2004-04-01      101.6
   Category 1960 1965 1970 1975 1980 1985 1990 1991 1992  ... 2012 2013 2014  \
13      NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  ...  NaN  NaN  NaN   
14      NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  ...  NaN  NaN  NaN   
15      NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  ...  NaN  NaN  NaN   
16      NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  ...  NaN  NaN  NaN   
17      NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  ...  NaN  NaN  NaN   
18      NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  ...  NaN  NaN  NaN   
19      NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  ...  NaN  NaN  NaN   
20      NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  ...  NaN  NaN  NaN   
21      NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  ...  NaN  NaN  NaN   
22      NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  Na

In [67]:
execute_function(df_list,missing_to_nan,'Missing to nan')

Missing to nan
                          Category  1960  1965  1970  1975  1980  1985  1990  \
1                           Truckb   NaN   NaN   NaN   NaN   NaN  5.34  7.64   
2                     Class I rail  1.40  1.27  1.43  2.04  2.87  3.04  2.66   
3   Water Transportation, domestic   NaN   NaN   NaN   NaN   NaN   NaN  1.42   
4                     Oil pipeline   NaN   NaN   NaN   NaN   NaN   NaN  1.22   
6                              NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   
7                              NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   
8                              NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   
9                              NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   
10                             NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   
11                             NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   
12                             NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   
13                       

                                        Category    1998    1999    2000  \
0                                 All industries  35,509  37,050  39,231   
1          Transportation and warehousing, total  35,965  37,426  39,055   
2                                            Air  48,919  50,879  53,231   
3                                           Rail  59,766  61,287  63,353   
4                                         Water   48,450  50,407  51,516   
5                                          Truck  33,686  34,864  36,030   
6    Transit and ground passenger transportation  21,840  22,801  23,827   
7                                      Pipeline   64,305  71,789  96,662   
8   Other transportation and support activitiesb  32,605  34,489  35,823   
9                        Warehousing and storage  31,317  32,307  33,623   
10                                           NaN     NaN     NaN     NaN   
11                                           NaN     NaN     NaN     NaN   
12          

[5 rows x 42 columns]


We have successfully transformed all the missing values in the consistent format in order to delete them.

In [68]:
def delete_na(df):
    df.dropna(thresh=2,inplace=True)
    df.dropna(thresh=2,axis='columns',inplace=True)
    print(df[df.isna().any(axis=1)])
   # Function deletes na in columns and rows who have at least 2 non missing values in df

In [69]:
def replace_regex(df,regex,colname,insert,newtype):
    df[colname] = df[colname].replace(regex,'',regex=True).astype(newtype)
# This function was created in order to make transformation of values faster. Function replaces the unnecessary signs or letters and converts to desired datatype.

In [70]:
execute_function(df_list,delete_na,'Deleting Na')

Deleting Na
                         Category  1960  1965  1970  1975  1980  1985  1990  \
1                          Truckb   NaN   NaN   NaN   NaN   NaN  5.34  7.64   
2                    Class I rail  1.40  1.27  1.43  2.04  2.87  3.04  2.66   
3  Water Transportation, domestic   NaN   NaN   NaN   NaN   NaN   NaN  1.42   
4                    Oil pipeline   NaN   NaN   NaN   NaN   NaN   NaN  1.22   

   1991  1992  ...   2012   2013   2014   2015   2016   2017   2018  \
1  7.54  7.75  ...  16.72  15.99  17.68  17.75  17.34  16.10  17.55   
2  2.59  2.58  ...   3.95   4.05   4.05   3.97   3.99   4.02   4.23   
3  1.41  1.39  ...   3.18   3.36   3.37   3.25   3.00   2.74   2.88   
4  1.18  1.21  ...   1.63   1.92   2.26   2.49   2.66   2.88   3.11   

        2019   2020  2021  
1  (R) 17.84  17.88   NaN  
2       4.42   4.40   NaN  
3       3.22   2.94   NaN  
4       3.41   3.44   NaN  

[4 rows x 39 columns]
                         Category   1960   1965   1970   1975   1980   19

Some of the datasets require transformation before proceeding to remove all NaN values without losing valuable data. From now on I will be inspecting and cleaning all the datasets one by one. Some of the tables require converting from wide to long format. I will rename columns containing year among the datasets to one consistent name.I will give descriptive names to the remaining columns.

In [71]:
df_list[0]

# dataset contains data about average freight revenue per tonemile over the years

,Category,1960,1965,1970,1975,1980,1985,1990,1991,1992,...,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021
0,"Air carrier, domestica",22.80,20.46,21.91,28.22,46.31,48.77,59.71,60.95,63.07,...,133.51,135.47,135.85,124.04,119.90,125.15,137.49,137.67,122.98,97.86
1,Truckb,NaN,NaN,NaN,NaN,NaN,5.34,7.64,7.54,7.75,...,16.72,15.99,17.68,17.75,17.34,16.10,17.55,(R) 17.84,17.88,NaN
2,Class I rail,1.40,1.27,1.43,2.04,2.87,3.04,2.66,2.59,2.58,...,3.95,4.05,4.05,3.97,3.99,4.02,4.23,4.42,4.40,NaN
3,"Water Transportation, domestic",NaN,NaN,NaN,NaN,NaN,NaN,1.42,1.41,1.39,...,3.18,3.36,3.37,3.25,3.00,2.74,2.88,3.22,2.94,NaN
4,Oil pipeline,NaN,NaN,NaN,NaN,NaN,NaN,1.22,1.18,1.21,...,1.63,1.92,2.26,2.49,2.66,2.88,3.11,3.41,3.44,NaN
5,Producer Price Index (1990 = 100)c,28,29,33,49,74,88,100.00,102.00,103.00,...,163.00,165.00,168.00,163.00,161.00,166.00,171.00,173,170.00,185


In [72]:
avg_fr_rev_per_tone_mile = pd.melt(df_list[0],id_vars=['Category'],var_name='Year',value_name='Avg_freight_rev_tonemile')
avg_fr_rev_per_tone_mile.dropna(inplace=True)

avg_fr_rev_per_tone_mile = avg_fr_rev_per_tone_mile[avg_fr_rev_per_tone_mile.Category == 'Truckb']
avg_fr_rev_per_tone_mile.drop(columns=['Category'],inplace=True)
replace_regex(avg_fr_rev_per_tone_mile,r'[(R)]','Avg_freight_rev_tonemile','',float)

# First dataset transformation

#Below list will contain transformed datasets. They will be converted to csv files after transformation.

df_list_to_csv = []
df_list_to_csv.append(avg_fr_rev_per_tone_mile)




In [73]:
avg_fr_rev_per_tone_mile.head()

,Year,Avg_freight_rev_tonemile
31,1985,5.34
37,1990,7.64
43,1991,7.54
49,1992,7.75
55,1993,8.48


I have decided to transform the table from wide to long format for better visualization. I have deleted the rows 
with values not providing information about truck transport. I have deleted all the rows where the 'Amount' was missing. I will perform similar transformation for the remaining datasets that require it.

In [74]:
avg_salary_transport = df_list[1]

avg_salary_transport.rename(columns={'(R) 2017':2017,'(R) 2018': 2018,'(R) 2019': 2019,'(R) 2020': 2020},inplace=True)
avg_salary_transport = pd.melt(avg_salary_transport, id_vars = ['Category'], var_name = 'Year', value_name = 'Avg_salary')
avg_salary_transport = avg_salary_transport[(avg_salary_transport.Category == 'All industries')|(avg_salary_transport.Category == 'Transportation and warehousing, total')|(avg_salary_transport.Category == 'Truck')]
replace_regex(avg_salary_transport,',','Avg_salary','',float)
avg_salary_transport.Year = avg_salary_transport.Year.astype(int)
df_list_to_csv.append(avg_salary_transport)
avg_salary_transport.head()


,Category,Year,Avg_salary
0,All industries,1998,35509.0
1,"Transportation and warehousing, total",1998,35965.0
5,Truck,1998,33686.0
10,All industries,1999,37050.0
11,"Transportation and warehousing, total",1999,37426.0


Above dataset contains information about average salary in different transport segments. I have decided to keep the data reflecting all industries, transport industry overall and trucking. It might be good idea to compare the trend between trucking and industry average over the years. Table was converted from wide to long format.

In [75]:
diesel_price = df_list[2]
diesel_price.head()

,DATE,GASDESW
0,1994-03-21,1.106
1,1994-03-28,1.107
2,1994-04-04,1.109
3,1994-04-11,1.108
4,1994-04-18,1.105


In [76]:

diesel_price.rename(columns={'GASDESW': 'Diesel_price_per_gallon'},inplace=True)
# Below function is made specifically for the datasets downloaded from fred.stlouisfed.ord due to their format
# The function converts the Date column to year value only and leaves only the first observation for each year.
def fred_dataset_cleaning(df):
    dateprice = []
    for i in df['DATE']:
        a = str(i)[:4]
        dateprice.append(a)
    df['DATE'] = dateprice
    df.drop_duplicates(subset=['DATE'],inplace=True)
    df.rename(columns={'DATE':'Year'},inplace=True)
fred_dataset_cleaning(diesel_price)
df_list_to_csv.append(diesel_price)
diesel_price

,Year,Diesel_price_per_gallon
0,1994,1.106
41,1995,1.104
93,1996,1.148
146,1997,1.291
198,1998,1.147
250,1999,0.965
302,2000,1.309
354,2001,1.522
407,2002,1.168
459,2003,1.501


Above dataset contains diesel price over the years. I will keep only year-to-year data. The first observation for each year remains in the table.

In [77]:
ppi_heavy_truck_man = df_list[3]
fred_dataset_cleaning(ppi_heavy_truck_man)
ppi_heavy_truck_man.rename(columns={'PCU336120336120P':'PPI_heavy_truck_manufacturing'},inplace=True)
df_list_to_csv.append(ppi_heavy_truck_man)
ppi_heavy_truck_man

,Year,PPI_heavy_truck_manufacturing
0,2003,100.000
1,2004,101.300
13,2005,104.400
25,2006,108.500
37,2007,113.700
49,2008,118.400
61,2009,123.700
73,2010,126.700
85,2011,131.100
97,2012,134.300


This dataset contains Producer price index for heavy truck manufacturing. I will keep only year-to-year data. The first observation for each year remains in the table.

In [78]:
ppi_truck_transportation = df_list[4]
ppi_truck_transportation.rename(columns={'PCU484484':'PPI_truck_transportation'},inplace=True)
fred_dataset_cleaning(ppi_truck_transportation)
df_list_to_csv.append(ppi_truck_transportation)
ppi_truck_transportation


,Year,PPI_truck_transportation
0,2003,100.000
1,2004,100.400
13,2005,106.000
25,2006,111.300
37,2007,113.800
49,2008,119.000
61,2009,118.500
73,2010,117.300
85,2011,122.200
97,2012,128.700


Above dataset contains Producer price index for truck transportation. I will keep only year-to-year data. The first observation for each year remains in the table.

In [79]:
total_operating_revenues = df_list[5]
ttl_operating_rev = pd.melt(total_operating_revenues, id_vars=['Category'],var_name='Year',value_name='Operating_revenues')
ttl_operating_rev.dropna(subset=['Operating_revenues'],inplace=True)
replace_regex(ttl_operating_rev,r'\D+','Operating_revenues','',float)
ttl_operating_rev = ttl_operating_rev[ttl_operating_rev['Category'] == 'Truckinga']
ttl_operating_rev.drop(columns=['Category'],inplace=True)
ttl_operating_rev.Year = ttl_operating_rev.Year.astype(int)
df_list_to_csv.append(ttl_operating_rev)
ttl_operating_rev.head()


,Year,Operating_revenues
79,1990,127314.0
92,1991,126772.0
105,1992,135437.0
118,1993,142547.0
131,1994,155713.0


Above dataset contains information about total operating revenues over the years. I have decided to keep only the values for trucking. Table was converted from wide to long format.

In [80]:
df_list[6]

,DATE,VAPGDPT
0,1/1/2005,2.8
1,4/1/2005,2.9
2,7/1/2005,2.9
3,10/1/2005,3.0
4,1/1/2006,3.0
...,...,...
65,4/1/2021,2.9
66,7/1/2021,3.0
67,10/1/2021,3.1
68,1/1/2022,3.1


In [81]:
transport_warehousing_percent_of_gdp = df_list[6]

transport_warehousing_percent_of_gdp.rename(columns={'VAPGDPT':'Transport_warehousing_p_of_gdp'},inplace=True)
def fred_date_reverse(df):
    date_list = []
    for i in transport_warehousing_percent_of_gdp['DATE']:
        a = str(i)[-4:]
        date_list.append(a)
    df['DATE'] = date_list
    df.drop_duplicates(subset=['DATE'],inplace=True)
    df.rename(columns={'DATE':'Year'},inplace=True)
# Function transforms date column into year and keeps only one observation per year, similar to fred_dataset_cleaning but with reversed year position in DATE column
    
fred_date_reverse(transport_warehousing_percent_of_gdp)
df_list_to_csv.append(transport_warehousing_percent_of_gdp)
transport_warehousing_percent_of_gdp.head()

,Year,Transport_warehousing_p_of_gdp
0,2005,2.8
4,2006,3.0
8,2007,2.9
12,2008,2.8
16,2009,2.8


This table contains transport and warehousing as percent of total gdp over the years. I will keep only year-to-year data. I keep only the first observation for each year.

In [82]:
truck_registration = df_list[7]
truck_reg = pd.melt(truck_registration,id_vars=['Category'],var_name='Year',value_name='Number_registered')
truck_reg.dropna(inplace=True)
truck_reg = truck_reg[truck_reg.Category != 'Farm trucks']
df_list_to_csv.append(truck_reg)
truck_reg.head()

,Category,Year,Number_registered
0,Total trucks,1995,64778472
1,Pickups,1995,28200171
2,Vans,1995,11557118
3,Sport utilities,1995,7440984
4,Other lighta,1995,9590256


Truck_reg dataset was converted from wide to long format and contains information about truck registration over the years. Some of the Na values were also dropped.


In [83]:
gdp_for_hire_transportation = df_list[8]
gdp_for_hire_transport = pd.melt(gdp_for_hire_transportation,id_vars=['Category'],var_name='Year',value_name='Gdp')
df_list_to_csv.append(gdp_for_hire_transport)
gdp_for_hire_transport.isna().any().sum() # It is 0
replace_regex(gdp_for_hire_transport,r',','Gdp','',float)

gdp_for_hire_transport

,Category,Year,Gdp
0,TOTAL U.S. GDP,1980,2788.10
1,"For-hire transportation services GDP, total",1980,102.60
2,Air transportation,1980,13.10
3,Rail transportation,1980,20.20
4,Water transportation,1980,3.50
...,...,...,...
805,Truck transportation,2021,28.59
806,Transit and ground passenger transportation,2021,6.61
807,Pipeline transportation,2021,6.64
808,Other transportation and support activities,2021,26.13


The data frame gdp_for_hire_transport provides information about gdp of different transportation services. The table was converted from wide to long format. The Gdp column has been transformed to float.

In [84]:
hazardous_materials_shipments = df_list[9]
hazardous_materials_shipments.drop(columns=['Unnamed: 2','Unnamed: 4','Unnamed: 6'],inplace=True)
hazardous_materials_shipments.dropna(inplace=True)

for col in hazardous_materials_shipments:
    if col != 'Category':
        replace_regex(hazardous_materials_shipments,r',',col,'',float)
hazardous_materials_shipments= hazardous_materials_shipments[(hazardous_materials_shipments['Category'] != 'For-hire')&(hazardous_materials_shipments['Category'] != 'Privateb')]
df_list_to_csv.append(hazardous_materials_shipments)
hazardous_materials_shipments.head()

,Category,Value,Tons,Ton-miles,Average miles per shipment
1,TOTAL all modes,1680.2,2968.0,382.5,189.0
2,"Single modes, total",1612.1,2889.5,307.2,72.0
3,Trucka,1091.3,1814.8,126.8,63.0
6,Rail,39.0,90.4,61.7,640.0
7,Water,137.1,304.2,60.9,72.0


The dataset hazardous_materials_shipments contains information about hazardous materials shipments distribution over different modes of transport.
The numeric values were converted to float. <br> <br>
Despite the fact that this is the only dataset not containing historical data, I have decided to include it in my analysis.

In [85]:
df_list[10]
ton_miles_freight = pd.melt(df_list[10],id_vars=['Categories'],var_name='Year',value_name='Ton_miles')
for col in ton_miles_freight:
    if col != 'Categories':
        replace_regex(ton_miles_freight,r'\D+',col,'',int)
ton_miles_freight1 = ton_miles_freight[(ton_miles_freight.Categories == 'Truck') | (ton_miles_freight.Categories == 'TOTAL U.S. ton-miles of freight ')]
df_list_to_csv.append(ton_miles_freight1)
ton_miles_freight1.head()

,Categories,Year,Ton_miles
0,TOTAL U.S. ton-miles of freight,1980,4207610
2,Truck,1980,1301607
12,TOTAL U.S. ton-miles of freight,1981,4230548
14,Truck,1981,1367507
24,TOTAL U.S. ton-miles of freight,1982,4122111


### Data Validation

Last of the datasets contains information about ton-miles of freight. I have decided to keep only the data about trucking and total transportation.
The dataset was converted from wide to long format. Numeric values were converted to int.

Lastly I will limit the measured data to the desired period (2006-2019) in every table.

In [86]:
new_df_list = []
def cut_years(df):
    df1 = df[(df['Year'].astype(int) > 2005)&(df['Year'].astype(int) < 2020)]
    return df1
for table in df_list_to_csv:
    if 'Year' in table.columns:
        new_table = cut_years(table)
        new_df_list.append(new_table)
    else:
        new_df_list.append(table)
# Function cuts the years in the table and returns dataframe after cut
# loop applies the function to all the data frames and puts new data frames in the list as well as the data frames for which the cutting was not necessary.
    
new_df_list


[     Year  Avg_freight_rev_tonemile
 133  2006                     13.40
 139  2007                     13.22
 145  2008                     15.96
 151  2009                     12.03
 157  2010                     14.30
 163  2011                     17.51
 169  2012                     16.72
 175  2013                     15.99
 181  2014                     17.68
 187  2015                     17.75
 193  2016                     17.34
 199  2017                     16.10
 205  2018                     17.55
 211  2019                     17.84,
                                   Category  Year  Avg_salary
 80                          All industries  2006     47731.0
 81   Transportation and warehousing, total  2006     45491.0
 85                                   Truck  2006     43069.0
 90                          All industries  2007     49895.0
 91   Transportation and warehousing, total  2007     47413.0
 95                                   Truck  2007     44305.0
 100      

As we can see the datasets are cleaned and ready to be exported for further analysis.

### Exporting the tables

In [87]:
file_names = ['avg_freight_revenue_per_tonemile','avg_salary_transportation','diesel_price','ppi_hd_truck_manufacturing',
             'ppi_truck_transportation','total_operating_revenues','transport_warehousing_percent_of_gdp','truck_registration',
             'gdp_for_hire_transportation','hazardous_materials_shipments','ton_miles_freight']
# avg_freight_revenue_per_tonemile = dataframes[0] 
# avg_salary_transportation = dataframes[1]
# diesel_price = dataframes[2]
# ppi_hd_truck_manufacturing = dataframes[3]
# ppi_truck_transportation = dataframes[4]
# total_operating_revenues = dataframes[5]
# transport_warehousing_percent_of_gdp = dataframes[6]
# truck_registration = dataframes[7]
# gdp_for_hire_transportation = dataframes[8]
# hazardous_materials_shipments = dataframes[9]
# ton_miles_freight = dataframes[10]
def export_table_csv(table_list, filenames):
    
    for i in range(len(table_list)):
        table_list[i].to_csv('.\\csv_cleaned\\' + filenames[i] + '.csv',index=False)
export_table_csv(new_df_list,file_names)

### Summary

All the above dataset contain relevant information. I have decided to keep year-to-year data only.
Datasets will be loaded into different notebook to perform eda. I have saved the datasets as a csv files in order to create a dashboard in Power BI. 